# Optimizing

Running faster your code.

## Measuring times

In [12]:
import numpy as np
a = np.arange(1000)
%timeit a**2

The slowest run took 26881.47 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.64 µs per loop


## Profiling

In [19]:
'{:.100f}'.format(pi)

'3.1415926535897931159979634685441851615905761718750000000000000000000000000000000000000000000000000000'